# Try Resnet

## Paras Setting

In [10]:
import sys, os
current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/redux/'
print(current_dir, DATA_HOME_DIR)

/home/ubuntu/fastai-notes/deeplearning1/nbs /home/ubuntu/fastai-notes/deeplearning1/nbs/data/redux/


In [14]:
from utils import *
%matplotlib inline

batch_size = 64
no_of_epochs = 3
path = DATA_HOME_DIR
model_path = path+'model'

## Use Reset

In [1]:
import resnet50
from resnet50 import Resnet50

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [3]:
# omit the last dense layer, so that we don't have to do model.pop
# before we finetune the model to suit our purpose
rn0 = Resnet50(include_top=False).model
rn0.output_shape[1:]

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


(2048, 7, 7)

```py
# from utils.py
def get_classes(path):
    batches = get_batches(path+'train', shuffle=False, batch_size=1)
    val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
    test_batches = get_batches(path+'test', shuffle=False, batch_size=1)
    return (val_batches.classes, batches.classes, onehot(val_batches.classes),
            onehot(batches.classes), val_batches.filenames, batches.filenames, 
            test_batches.filenames)
```

In [11]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
# labels are the one-hot encoded version of classes
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [12]:
val_features = rn0.predict_generator(val_batches, val_batches.nb_sample)

In [13]:
trn_features = rn0.predict_generator(batches, batches.nb_sample)

```py
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
```

In [15]:
save_array(model_path + 'trn_rn0_conv.bc', trn_features)
save_array(model_path + 'val_rn0_conv.bc', val_features)

In [18]:
trn_features = load_array(model_path + 'trn_rn0_conv.bc')
val_features = load_array(model_path + 'val_rn0_conv.bc')

### FC net

In [19]:
def get_fc_layers(p):
    return [
        BatchNormalization(axis=1, input_shape=rn0.output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [20]:
model = Sequential(get_fc_layers(.5))

In [21]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(trn_features, trn_labels, nb_epoch=2,
         batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/2
23000/23000 [==============================] - 41s - loss: 0.1064 - acc: 0.9724 - val_loss: 0.0603 - val_acc: 0.9825
Epoch 2/2
23000/23000 [==============================] - 41s - loss: 0.0320 - acc: 0.9897 - val_loss: 0.0562 - val_acc: 0.9855


### Global average pooling

In [24]:
def get_ap_layers(p):
    return [
        GlobalAveragePooling2D(input_shape=rn0.output_shape[1:]),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [25]:
model = Sequential(get_ap_layers(0.2))

In [26]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(trn_features, trn_labels, nb_epoch=3,
         batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 6s - loss: 0.0656 - acc: 0.9761 - val_loss: 0.0362 - val_acc: 0.9875
Epoch 2/3
23000/23000 [==============================] - 6s - loss: 0.0360 - acc: 0.9868 - val_loss: 0.0354 - val_acc: 0.9880
Epoch 3/3
23000/23000 [==============================] - 6s - loss: 0.0301 - acc: 0.9894 - val_loss: 0.0304 - val_acc: 0.9890


### Use Resnet Large

In [29]:
rn1 = Resnet50(include_top=False, size=(400, 400)).model
rn1.output_shape[1:]

/home/ubuntu/anaconda2/envs/py36/lib/python3.6/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 400, 400)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


(2048, 13, 13)

In [32]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size,
                     target_size=(400, 400))
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False,
                         target_size=(400, 400))
# labels are the one-hot encoded version of classes
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [33]:
val_features = rn1.predict_generator(val_batches, val_batches.nb_sample)
trn_features = rn1.predict_generator(val_batches, val_batches.nb_sample)

In [34]:
save_array(model_path + 'trn_rn1_conv.bc', trn_features)
save_array(model_path + 'val_rn1_conv.bc', val_features)

In [35]:
trn_features = load_array(model_path + 'trn_rn1_conv.bc')
val_features = load_array(model_path + 'val_rn1_conv.bc')

In [ ]:
def get_ap_layers(p):
    return [
        GlobalAveragePooling2D(input_shape=rn1.output_shape[1:]),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [36]:
model.fit(trn_features, trn_labels, nb_epoch=3,
         batch_size=batch_size, validation_data=(val_features, val_labels))

ValueError: Error when checking model input: expected globalaveragepooling2d_input_1 to have shape (None, 2048, 7, 7) but got array with shape (2000, 2048, 13, 13)